## Requirements

- TTS
- cutlet
- unidic
- SpeechRecognition

In [ ]:
pip install TTS

In [ ]:
pip install TTS[ja]

In [ ]:
pip install cutlet

In [ ]:
pip install 'fugashi[unidic]'

In [ ]:
!python -m unidic download

In [ ]:
pip install SpeechRecognition

# Imports

In [8]:
from TTS.api import TTS
import torch
import cutlet
import speech_recognition as sr

# TTS (Coqui)

Github ref: https://github.com/coqui-ai/TTS?tab=readme-ov-file

Models: https://github.com/coqui-ai/TTS/blob/dev/TTS/.models.json

I'm currently using multilingual multi-dataset. I'd recommend we finetune the model before using it to production because some phrases are not that clear or sound robotic. Some tasks I will still do:

- [ ] Find a voice database in english (probably I'll stick to mitsuru_kirijo or some RPG/famous movie voice)
- [ ] Find a voice database in portuguese and japanese
- [ ] Test other models for testing computation run time (currently 40s)
- [ ] Train/finetune the choosen model (optional: is it worth to train an F5-tts?)
- [ ] Make this an API (colab is just to prototyping quickly)

Notes:
- english works just fine, but sometimes it makes some mistakes
- japanese sometimes is not really smooth here I think is due to the reference audio I provided, maybe using a better sample will solve it



In [ ]:
# This should be in the initialize() and the variables must be globally declared

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# List available 🐸 TTS models
print(TTS().list_models())

# Init 🐸 TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

# Currently available languages list
languages = {"en": "English", "ja": "Japanese", "pt": "Portuguese"}

# Change the japanese reference audio later (this sample is simply awful haha)
reference_audio_paths = {"en": "./mitsuru_kirijo_01.wav", "ja": "./bocchi_noisesV4.wav"}

# Speech Recognition
# API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
# obtain audio from the wav file
r = sr.Recognizer()

# OpenAI not implemented yet

In [12]:
# Choose language
lang = "en"

text = "The greatest joy of magic lies in searching for it"

In [ ]:
# This should be a route

def textToSpeech(text, lang):
  # Error handling
  if lang not in languages.keys():
      print("Language not supported")
      exit()

  # Generate TTS with voice cloning
  tts.tts_to_file(text=text, speaker_wav=reference_audio_paths[lang], file_path="test.wav", language = lang)
  return

# test
textToSpeech(text, lang)

# STT
It worked haha prob we'll need a Google API Key for production but I think STT won't be a short term bottleneck.

Next step is to get this text, send to OpenAI and generate a prompt-based answer to be send to TTS. Once this is achieved, we'll have a really basic SST model.

In [ ]:
# This should be another route

def speechToText(lang, audio):
  # Need to implement lang and mapping to the correct format (en -> en-US ...)
  # For now, only english is available or any other hardcoded lang (already tested japanese -> ja-JP and it works smoothly)
  with sr.AudioFile(f"{audio}.wav") as source:
      audio = r.record(source)  # read the entire audio file

  # recognize speech using Google Speech Recognition
  try:
      # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
      # instead of `r.recognize_google(audio)`
      print("Google Speech Recognition thinks you said " + r.recognize_google(audio, language="en-US"))
      return r.recognize_google(audio, language="en-US")
  except sr.UnknownValueError:
      print("Google Speech Recognition could not understand audio")
  except sr.RequestError as e:
      print("Could not request results from Google Speech Recognition service; {0}".format(e))

# test
test_text = speechToText("en", "test")